In [ ]:
import os
import sys
src_path = os.path.abspath('../src')
sys.path.append(src_path)
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd
import numpy as np

from src import data
from src import portfolio
from src import efficient_frontier
from src import plot
from src import capm
import yfinance as yf
from scipy.optimize import Bounds, LinearConstraint
import statsmodels.api as smf

In [ ]:
weights = np.array([1/3,1/3,1/3])
min_esg_score = 1100
max_esg_score = 2000
df = pd.read_excel(r"../data/ESG_DATA_S&P500.xlsx")

dates = ['2021-01-01','2022-01-01']
start_year = dates[0]
end_year = dates[1]

Bounds = Bounds(0,1) #How willing we are to go short and to invest in one particular stock
Wanted_return = 0.20 #Only used when using the wanted_return constraint
maximum_risk = 0.10 #Either used when using maximum risk constraint or cmle portfolio
sharpe_type = "No_extra_constraint"  # rename to constraint, can be either Wanted_return or Maximum_risk or No_extra_constraint
risk_free_rate = 0.02

In [ ]:
esg_data = data.esg_score_weight(df,weights,min_esg_score) #ESG_DATA filtering function
full_data = data.stock_monthly_close(esg_data,dates) #Gives us the data we need from the stocks with relevant ESG_scores
prices,esgdata = data.seperate_full_data(full_data) #Gives us price data and esg data to use in future functions
prices = prices[['MMM','AOS','ABT','ACN','ADM','ADP']]
esgdata = esgdata[['MMM_weighted','AOS_weighted','ABT_weighted','ACN_weighted','ADM_weighted','ADP_weighted']]

pct_returns = data.pct_returns_from_prices(prices) #Gives us the returns in pct for our stocks
parameters = portfolio.efficient_frontier_solo(pct_returns,Bounds, sharpe_type,dates[0],dates[1], Wanted_return, maximum_risk) #Different amount of rows from ESG_DATA as it drops all na. Gives us the efficient frontier for the given tie frame
weights_of_each_portfolio = efficient_frontier.weights_of_portfolio(prices,parameters)
sp500 = data.data_for_beta(['SPY','MMM','AOS','ABT','ACN','ADM','ADP'],dates)
pct_returns_sp500 = data.pct_returns_from_prices(sp500)


In [ ]:
    
betas_of_portfolio = capm.calculate_portfolio_beta(pct_returns_sp500,prices,weights_of_each_portfolio)
betas_of_portfolio


In [ ]:

capm.capm_calc(pct_returns_sp500['SPY'].mean(),risk_free_rate,betas_of_portfolio)



In [ ]:
capm.calculate_portfolio_beta_ols(pct_returns_sp500,prices,weights_of_each_portfolio)